In [ ]:
%load_ext autoreload
%autoreload

import getpass
import common
import maxcut
import dwave.inspector
import csv
from itertools import count

### Constants

In [ ]:
SEPERATOR = "--------------------------------------------------"
path = "../../_common/instances/mc_024_003_000.txt"
outputFile = "maxcut_results.csv"

In [ ]:
# Get key to use for D-Wave Call
prompt = "" 
token = ""
try:
    prompt = getpass.getpass(prompt="D-Wave Key: ", stream=None) 
except Exception as error:
    print('ERROR', error)
else:
    print("Key Accepted")

In [ ]:
# Load nodes and edges
nodes, edges = common.read_maxcut_instance(path)

# Simulated Annealing

In [ ]:
result = maxcut.maxcut_sa(nodes, edges, 5000)
print(result)

# Quantum Annealing

In [ ]:
embedding = None
shots = 100

with open(outputFile, "w") as f:

     optimal = common.optimal_maxcut(path)

     writer = csv.writer(f)
     writer.writerow(["file", "shots", "qpu_anneal_time_per_sample", "max", "optimal", "mean", "sd",
          "qpu_sampling_time", "qpu_readout_time_per_sample", "qpu_access_time", \
          "qpu_access_overhead_time", "qpu_programming_time", "qpu_delay_time_per_sample", "qpu_programming_time", \
          "total_post_processing_time", "post_processing_overhead_time", "wallclockStart", "wallclockEnd", "embedding", "wallTime"])

     for i in range(8):
          x = 2**i
          if x > 200:
               break

          embeddingCreated = True if embedding==None else False
          print ("Created " if embeddingCreated else "Reused ", "Embedding")

          sampleset, result, embedding = maxcut.maxcut_qa(nodes, edges, shots, token, x, embedding)

          # sample to file
          timing = sampleset.info["timing"]
          data = [path, shots, timing["qpu_anneal_time_per_sample"], result["stats"]["max"], optimal, result["stats"]["mean"], result["stats"]["sd"],
               timing["qpu_sampling_time"], timing["qpu_readout_time_per_sample"], timing["qpu_access_time"], \
               timing["qpu_access_overhead_time"], timing["qpu_programming_time"], timing["qpu_delay_time_per_sample"], timing["qpu_programming_time"], \
               timing["total_post_processing_time"], timing["post_processing_overhead_time"], result["stats"]["wallClockStart"], result["stats"]["wallClockEnd"], \
               "Created" if embeddingCreated else "Reused", result["stats"]["wallClockEnd"] - result["stats"]["wallClockStart"] ]

          writer.writerow(data)

In [ ]:
print(result["stats"])

print(sampleset.info)
print(SEPERATOR)
timing = sampleset.info["timing"]
data = [timing["qpu_anneal_time_per_sample"], result["stats"]["max"], result["stats"]["mean"], result["stats"]["sd"],
    timing["qpu_sampling_time"], timing["qpu_readout_time_per_sample"], timing["qpu_access_time"], \
    timing["qpu_access_overhead_time"], timing["qpu_programming_time"], timing["qpu_delay_time_per_sample"], timing["qpu_programming_time"], \
    timing["total_post_processing_time"], timing["post_processing_overhead_time"]]

print (data)

In [ ]:
sampleset, result, embedding = maxcut.maxcut_qa(nodes, edges, 5000, token, 1, None)

In [ ]:
sampleset, result, a = maxcut.maxcut_qa(nodes, edges, 5000, token, 16, embedding)

# sample to file
timing = sampleset.info["timing"]
data = [path, shots, timing["qpu_anneal_time_per_sample"], result["stats"]["max"], optimal, result["stats"]["mean"], result["stats"]["sd"],
    timing["qpu_sampling_time"], timing["qpu_readout_time_per_sample"], timing["qpu_access_time"], \
    timing["qpu_access_overhead_time"], timing["qpu_programming_time"], timing["qpu_delay_time_per_sample"], timing["qpu_programming_time"], \
    timing["total_post_processing_time"], timing["post_processing_overhead_time"], result["stats"]["wallClockStart"], result["stats"]["wallClockEnd"], \
    "Created" if embeddingCreated else "Reused", result["stats"]["wallClockEnd"] - result["stats"]["wallClockStart"] ]

writer.writerow(data)